In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/test_2umaH9m.csv
/kaggle/input/sample_submission_M0L0uXE.csv
/kaggle/input/train_LZdllcl.csv


In [3]:
import pandas as pd
df_train = pd.read_csv('/kaggle/input/train_LZdllcl.csv')
df_test = pd.read_csv('/kaggle/input/test_2umaH9m.csv')
df_sub = pd.read_csv('/kaggle/input/sample_submission_M0L0uXE.csv')


In [4]:
import math
median_rate = math.floor(df_train['previous_year_rating'].median())
df_train['previous_year_rating'] = df_train['previous_year_rating'].fillna(median_rate)

In [5]:
df_train.isna().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating       0
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [6]:
df_test.isna().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [7]:
import math
median_rate = math.floor(df_test['previous_year_rating'].median())
df_test['previous_year_rating'] = df_test['previous_year_rating'].fillna(median_rate)

In [8]:
df_test.isna().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating       0
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df_train  = df_train.drop('education',axis=1)

In [13]:
df_train['department'] = le.fit_transform(df_train['department'])
df_train['region'] = le.fit_transform(df_train['region'])
df_train['gender'] = le.fit_transform(df_train['gender'])
df_train['recruitment_channel'] = le.fit_transform(df_train['recruitment_channel'])

In [14]:
df_train.head()

,employee_id,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,7,31,0,2,1,35,5.0,8,1,0,49,0
1,65141,4,14,1,0,1,30,5.0,4,0,0,60,0
2,7513,7,10,1,2,1,34,3.0,7,0,0,50,0
3,2542,7,15,1,0,2,39,1.0,10,0,0,50,0
4,48945,8,18,1,0,1,45,3.0,2,0,0,73,0


In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [16]:
df_train = df_train.drop('employee_id',axis=1)

In [17]:
X = df_train.iloc[:,0:11]
Y = df_train.iloc[:,-1]

In [20]:
best_features = SelectKBest(score_func = chi2, k=10)
fit = best_features.fit(X,Y)

In [21]:
dfscore = pd.DataFrame(fit.scores_)
dfcolumn = pd.DataFrame(X.columns)
featurescore = pd.concat([dfcolumn,dfscore],axis=1)
featurescore.columns = ['Specs','Score']

In [22]:
featurescore

,Specs,Score
0,department,0.001172
1,region,21.332612
2,gender,2.013113
3,recruitment_channel,0.301599
4,no_of_trainings,10.063491
5,age,27.228103
6,previous_year_rating,574.656810
7,length_of_service,19.351084
8,KPIs_met >80%,1743.827117
9,awards_won?,2054.009313


In [23]:
print(featurescore.nlargest(11,'Score'))

                   Specs        Score
10    avg_training_score  5072.973743
9            awards_won?  2054.009313
8          KPIs_met >80%  1743.827117
6   previous_year_rating   574.656810
5                    age    27.228103
1                 region    21.332612
7      length_of_service    19.351084
4        no_of_trainings    10.063491
2                 gender     2.013113
3    recruitment_channel     0.301599
0             department     0.001172


In [24]:
X = X.drop(['department','recruitment_channel','gender','no_of_trainings','length_of_service','region','age'],axis=1)

In [25]:
X

,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score
0,5.0,1,0,49
1,5.0,0,0,60
2,3.0,0,0,50
3,1.0,0,0,50
4,3.0,0,0,73
...,...,...,...,...
54803,3.0,0,0,78
54804,2.0,0,0,56
54805,5.0,1,0,79
54806,1.0,0,0,45


In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_tets = train_test_split(X,Y,test_size = 0.2)

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(x_train,y_train)
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
model.score(x_test,y_tets)

0.9163473818646233

In [31]:
df_sub.shape

(23490, 2)

In [32]:
df_test

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,3.0,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23485,53478,Legal,region_2,Below Secondary,m,sourcing,1,24,3.0,1,0,0,61
23486,25600,Technology,region_25,Bachelor's,m,sourcing,1,31,3.0,7,0,0,74
23487,45409,HR,region_16,Bachelor's,f,sourcing,1,26,4.0,4,0,0,50
23488,1186,Procurement,region_31,Bachelor's,m,sourcing,3,27,3.0,1,0,0,70


In [33]:
df_test = df_test.drop(['department','recruitment_channel','gender','no_of_trainings','length_of_service','region','age','employee_id'],axis=1)

In [35]:
df_test = df_test.drop('education',axis=1)

In [36]:
pred = model.predict(df_test)

In [38]:
pred.shape

(23490,)

In [40]:
res = pred.tolist()

In [43]:
df_sub = df_sub.drop('is_promoted',axis=1)

In [45]:
df_sub['is_promoted'] = res

In [47]:
df_sub.to_csv('result101.csv',index=False)

In [50]:
X.shape

(54808, 4)

In [54]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
model_cnn = Sequential()
model_cnn.add(Dense(units = 100 , activation = 'relu' , input_dim = X.shape[1]))
model_cnn.add(Dense(units = 50 , activation = 'relu'))
model_cnn.add(Dense(units = 25 , activation = 'relu'))
model_cnn.add(Dense(units = 10 , activation = 'relu'))
model_cnn.add(Dense(units = 1 , activation = 'sigmoid'))

In [55]:
model_cnn.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [58]:
model_cnn.fit(x_train,y_train , epochs = 10)

Epoch 1/10
43846/43846 [==============================] - 2s 50us/step - loss: 0.2618 - accuracy: 0.9142
Epoch 2/10
43846/43846 [==============================] - 2s 50us/step - loss: 0.2505 - accuracy: 0.9158
Epoch 3/10
43846/43846 [==============================] - 2s 51us/step - loss: 0.2440 - accuracy: 0.9170
Epoch 4/10
43846/43846 [==============================] - 2s 51us/step - loss: 0.2429 - accuracy: 0.9170
Epoch 5/10
43846/43846 [==============================] - 2s 50us/step - loss: 0.2407 - accuracy: 0.9170
Epoch 6/10
43846/43846 [==============================] - 2s 50us/step - loss: 0.2393 - accuracy: 0.9177
Epoch 7/10
43846/43846 [==============================] - 2s 50us/step - loss: 0.2390 - accuracy: 0.9177
Epoch 8/10
43846/43846 [==============================] - 2s 49us/step - loss: 0.2379 - accuracy: 0.9180
Epoch 9/10
43846/43846 [==============================] - 2s 51us/step - loss: 0.2358 - accuracy: 0.9186
Epoch 10/10
43846/43846 [==============================

In [62]:
pred = model_cnn.predict(x_test)

In [64]:
for i in range(len(pred)):
    if(pred[i] > 0.5):
        pred[i] = 1
    else:
        pred[i] = 0
        
from sklearn.metrics import accuracy_score 
accuracy_score(pred,y_tets)

0.917989417989418

In [68]:
res = model_cnn.predict(df_test)

In [70]:
for i in range(len(res)):
    if(res[i] > 0.5):
        res[i] = 1
    else:
        res[i] = 0
        


In [79]:
result = res.tolist()

In [80]:
df_sub = df_sub.drop('is_promoted',axis=1)

In [81]:
df_sub['is_promoted'] = result

ValueError: setting an array element with a sequence.